# Technical indicators coding

## Fibonacci retracement levels
The Fibonacci retracement levels are commonly used in technical analysis to identify potential support and resistance levels in a financial time series. These levels are calculated based on the Fibonacci sequence, which is a series of numbers where each number is the sum of the two preceding ones, starting from 0 and 1. The Fibonacci retracement levels are typically set at 23.6%, 38.2%, 50%, 61.8%, and 100% of the move from the high to low point in a given time series.

To calculate the Fibonacci retracement levels in a window of OHLCV data, you can use the following steps:

Identify the high and low points in the window: In OHLCV data, the high and low prices are typically represented by the "High" and "Low" columns, respectively. To identify the high and low points in the window, you can use the max and min functions on these columns.

Calculate the move from the high to low point: The move from the high to low point in the window is the difference between the high and low prices. You can calculate this using the diff function.

Calculate the Fibonacci retracement levels: Once you have the move from the high to low point, you can calculate the Fibonacci retracement levels using the following formula:

23.6% retracement level = high - (move * 0.236)
38.2% retracement level = high - (move * 0.382)
50% retracement level = high - (move * 0.5)
61.8% retracement level = high - (move * 0.618)
100% retracement level = high
Where high is the highest price in the window, and move is the difference between the high and low prices.

Here's an example code that shows how to calculate Fibonacci retracement levels in a window of OHLCV data using Pandas:

In [8]:
import os
import gc
import pandas as pd

def read_data(pair, start_date, end_date, data_dir, interval):
    """
    Reads data for a given trading pair and date range.
    
    Args:
        pair (str): Trading pair (e.g. 'BTCUSDT')
        start_date (pd.Timestamp): Start date of data to read
        end_date (pd.Timestamp): End date of data to read
        data_dir (str): Directory where data is stored
        interval (str): Interval of data to read (e.g. '1m', '1h', '1d')
    
    Returns:
        pd.DataFrame: DataFrame containing data for the given pair and date range
    """
    data = pd.DataFrame()
    
    # Iterate over all months between start_date and end_date
    for year in range(start_date.year, end_date.year + 1):
        for month in range(1, 13):
            gc.collect()
            if year == start_date.year and month < start_date.month:
                continue
            if year == end_date.year and month > end_date.month:
                continue
                
            # Construct file path
            file_path = os.path.join(data_dir, interval, pair, f"{pair}_{year}_{str(month).zfill(2)}.zip")
            
            # Check if file exists
            if os.path.exists(file_path):
                # Read data from file
                df= pd.read_csv(file_path, compression='zip', header=0, sep=',', quotechar='"', names=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
                # Filter data by date range
                mask = (df['Open time'] >= start_date.timestamp() * 1000) & (df['Open time'] <= end_date.timestamp() * 1000)
                data = pd.concat([data, df.loc[mask]])
    return data


In [2]:
import pandas as pd

# Load the OHLCV data
data = read_data("ETHUSDT", pd.Timestamp("2022-01-01"),  pd.Timestamp("2023-02-28"), "./database/Trades/", "1h")
# Set the window size
window_size = 20

# Define a function to calculate the Fibonacci retracement levels in a window
def calculate_fibonacci_retracement_levels(data):
    # Calculate the high and low points in the window
    high = data['High'].max()
    low = data['Low'].min()
    
    # Calculate the move from the high to low point
    move = high - low
    
    # Calculate the Fibonacci retracement levels
    retracement_23 = high - (move * 0.236)
    retracement_38 = high - (move * 0.382)
    retracement_50 = high - (move * 0.5)
    retracement_61 = high - (move * 0.618)
    retracement_100 = high
    
    return {
        '23.6%': retracement_23,
        '38.2%': retracement_38,
        '50%': retracement_50,
        '61.8%': retracement_61,
        '100%': retracement_100
    }

# Apply the function to each window of data
fibonacci_levels = data.rolling(window_size).apply(calculate_fibonacci_retracement_levels, raw=True)

# Print the results
print(fibonacci_levels)


Empty DataFrame
Columns: []
Index: []


In [7]:
read_data("BTCUSDT", pd.Timestamp("2022-01-01"),  pd.Timestamp("2023-02-28"), "./database/Trades/", "1h")

""


In [ ]:
import pandas as pd
import mplfinance as mpf

# Load the OHLCV data
data = pd.read_csv('ohlc_data.csv')
data = data.set_index('Date')

# Plot the candlestick chart
mpf.plot(data, type='candle', mav=(10, 20), volume=True)
